In [ ]:
import os
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import HumanMessage, SystemMessage
from langchain_community.vectorstores import FAISS
from configs import const

# OpenAI APIキーを環境変数から設定
os.environ["OPENAI_API_KEY"] = const.OPENAI_API_KEY

# ChatOpenAIのインスタンスを作成
llm = ChatOpenAI(temperature=0, model_name="gpt-4")

# ベクターストアの読み込み
embeddings = OpenAIEmbeddings()
db = FAISS.load_local("./faiss_index", embeddings, allow_dangerous_deserialization=True)

# システムプロンプトを定義
system_prompt = """
# 役割
あなたは親切なアシスタントです。
お客様の質問に丁寧な言い回しで回答してください。
"""

# 質問に対する回答を生成する関数
def generate_response(messages):
    user_question = messages[-1].content  # ユーザーの最新の質問を取得
    docs = db.similarity_search(user_question, k=5)  # ベクターストアから関連するドキュメントを取得

    # ドキュメントのコンテンツを抽出
    context = "\n\n".join([doc.page_content for doc in docs])
    context_prompt = f"以下の情報を参考にしてください:\n{context}\n\n"
    full_prompt = context_prompt + system_prompt + user_question

    response = llm([HumanMessage(content=full_prompt)])
    return response.content

# Streamlitアプリケーションの表示
def show():
    st.title("Insight AI")

    # セッションステートにメッセージを保存
    if "messages" not in st.session_state:
        st.session_state.messages = []
        # 初期化時にシステムプロンプトをメッセージリストに追加
        st.session_state.messages.append({"role": "system", "content": system_prompt})

    # メッセージの表示（システムプロンプトは表示しない）
    for message in st.session_state.messages:
        if message["role"] != "system":
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

    # ユーザーの入力を受け取る
    prompt = st.chat_input("文章を入力してください。")
    if prompt:
        with st.chat_message("user"):
            st.markdown(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})

        # メッセージリストにシステムプロンプトを含める
        messages = [SystemMessage(content=system_prompt)] + [
            HumanMessage(content=msg["content"]) for msg in st.session_state.messages if msg["role"] != "system"
        ]

        # レスポンスを生成
        response = generate_response(messages)
        st.session_state.messages.append({"role": "assistant", "content": response})

        with st.chat_message("assistant"):
            st.markdown(response)

if __name__ == "__main__":
    show()


Fetching: https://doomo.jp/
Fetching: https://doomo.jp/event/holiday-business
Fetching: https://doomo.jp/event/sdgs
Fetching: https://doomo.jp/event/engineer
Fetching: https://doomo.jp//
Fetching: https://doomo.jp/event/fudosan-baibai
Fetching: https://doomo.jp/event/writer-designer
Fetching: https://doomo.jp/wp-content/uploads/2022/10/220602freelance5.png
Fetching: https://doomo.jp/wp-content/uploads/2022/11/220906shigyofudosan4846.jpg
Fetching: https://doomo.jp/event/tax-accountant-maching
Fetching: https://doomo.jp/event/fudosan-koryukai
Fetching: https://doomo.jp/place/natuluck_kayabacho
Fetching: https://doomo.jp/event/shigyo-fudosan
Fetching: https://doomo.jp/event/hr
Fetching: https://doomo.jp/wp-content/uploads/2022/10/230124hr6778.jpg
Fetching: https://doomo.jp/event/young-manager
Fetching: https://doomo.jp/event/fudosan-kensetsu
Fetching: https://doomo.jp/event/shigotano
Fetching: https://doomo.jp/wp-content/uploads/2022/03/220721shigyofusan4360.jpg
Fetching: https://doomo.jp

In [2]:
!pip install faiss-cpu 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 21.5 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import requests
from bs4 import BeautifulSoup
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import dotenv
from configs import const

# 環境変数からAPIキーを読み込む
dotenv.load_dotenv()

# OpenAI APIキーを環境変数から設定
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# スクレイピング関数
def fetch_website_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        texts = soup.get_text(separator="\n", strip=True)
        return texts
    return ""

# 複数のURLを定義
urls = [
    "https://doomo.jp/", # Doomoのトップページ
    "https://doomo.jp/faq", # よくあるご質問について記載されている
    "https://doomo.jp/event", # 開催予定のイベント一覧【2024年】について記載されています。
    "https://doomo.jp/voc", # イベント参加者の皆様の声が記載されています。
    "https://doomo.jp/prohibited-matters", # 禁止事項について記載されています。
]

# 各URLからコンテンツを取得し、ドキュメントを作成
documents = []
for url in urls:
    content = fetch_website_content(url)
    if content:
        # テキストデータの保存
        if not os.path.exists('data'):
            os.makedirs('data')
        file_path = const.DATA_PATH + os.path.basename(url) + "_content.txt"
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)

        # ドキュメントの作成
        document = Document(page_content=content, metadata={"source": url})
        documents.append(document)

# テキストデータを小さく分割
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# ベクターDBの作成
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(splits, embeddings)
db.save_local(const.DATA_PATH + "faiss_index")


/Users/takuma.fukuda/Library/Caches/pypoetry/virtualenvs/mare-demo-dJVLw4eI-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
